In [1]:
import numpy as np
import xarray as xr

# Forecast Data for Winter 2022
The below data set is ECMWF's forecast data for winter 2022, produced in September 2022 (DJF with a lead time of 4, 5 and 6 months respectively)

In [2]:
XRDAT_LM06 = xr.open_dataset("C:/Users/durka/Downloads/Thesis_Data/WIN22_LM06.nc")

Latitude and longitude of Dublin Airport:

In [3]:
DUBLATLON = np.array([53.428, -6.241])

The code below extracts all 51 ensemble members' predictions at the nearest gridpoint to Dublin Airport for December, January and February.

In [4]:
DUBFC_MULTI = XRDAT_LM06.t2m.sel(latitude=xr.DataArray([DUBLATLON[0]], dims='lat'),
                                 longitude=xr.DataArray([DUBLATLON[1]], dims='lon'),
                                 method='nearest')

The issue now centres around choosing which ensemble member to choose. The one that will be chosen is the one which remains closest to the mean of the ensemble members over the three month period.

* DUBFC_MULTI_MEAN is an array of three elements, those being the mean ensemble prediction for Dec, Jan and Feb.
* DUBFC_MULTI_DEV is a matrix with time (Dec, Jan, Feb) as its rows and how far each ensemble member's predictions deviates from the mean as its columns.
* The first column of DUBFC_MULTI_DEV is a column vector of three elements. It represents how much ensemble member 1 deviates from the Dec mean, Jan mean and Feb mean. The second column vector represents how much ensemble member 2 deviates from the each month's mean etc.
* DUBFC_MULTI_AVGERR is an array of 51 elements, these elements being the norms of the column vectors described in the bullet point above.
* The ensemble member with the lowest average error (minimum of DUBFC_MULTI_AVGERR) is the member that deviates from the mean the least, and will be chosen for obtaining the forecasted temperature for winter 2022.

In [5]:
DUBFC_MULTI_MEAN = np.zeros(3)

for t in range(3):
    DUBFC_MULTI_MEAN[t] = np.mean(DUBFC_MULTI[t])
    
DUBFC_MULTI_DEV = np.zeros([3,51])
    
for t in range(3):
    for i in range(51):
        DUBFC_MULTI_DEV[t,i] = DUBFC_MULTI[t,i] - DUBFC_MULTI_MEAN[t]
    
DUBFC_MULTI_AVGERR = np.zeros(51)

for i in range(51):
    DUBFC_MULTI_AVGERR[i] = np.linalg.norm(DUBFC_MULTI_DEV[:,i])
    
MEANENSEMBLE = np.argmin(DUBFC_MULTI_AVGERR)
MEANENSEMBLE

14

The output of the above cell of code tells us that ensemble member 14 is the ensemble member that deviates from the mean the least.

Finally, we can obtain ensemble member 14's prediction for the mean temperature of winter 2022 (in ºC):

In [6]:
WIN2022FC_t2m = np.mean(DUBFC_MULTI[:,MEANENSEMBLE,0,0] - 273.15)
WIN2022FC_t2m

<xarray.DataArray 't2m' ()>
array(6.5438232, dtype=float32)
Coordinates:
    longitude  float32 -6.0
    latitude   float32 53.0
    number     int32 14

We can see that the forecasted average temperature for winter 2022 = 6.544ºC

# Climatology Prediction for Winter 2022
Begin by importing the monthly climatology predictions, obtained from the "Climatology" notebook

In [12]:
%store -r CLM_t2m
CLM_t2m[0,11]

5.571774193548387

Climatology prediction for a given winter is the average of the DJF predictions:

In [13]:
WIN2022CLM_t2m = (CLM_t2m[0,0]+CLM_t2m[0,1]+CLM_t2m[0,11])/3
WIN2022CLM_t2m

5.381274178737521

We can see that the average temperature for winter 2022, predicted by climatology = 5.381ºC

# Difference Between Forecast and Climatology Predictions

In [14]:
WIN2022FC_t2m - WIN2022CLM_t2m

<xarray.DataArray 't2m' ()>
array(1.16254906)
Coordinates:
    longitude  float32 -6.0
    latitude   float32 53.0
    number     int32 14

This result shows us that the earliest forecast for winter 2022 at Dublin Airport produced, produced by the ECMWF in September 2022, predicted that temperatures would be 1.163ºC warmer than average. This is the figure used in section 1.1 of my report.